## Create Hard Dataset

I have manually curated 100 **FAKE** videos that I believe are fairly hard for a human to identify that there is a deepfake present in the video.

Most of these videos contain a single person and are in a well-lit environment.

In [25]:
import os
import cv2
import glob
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from pathlib import Path
from os import listdir
from tqdm.notebook import tqdm

from EasyBlazeFace import EasyBlazeFace
from EasyRetinaFace import EasyRetinaFace

In [2]:
# Join metadata files into single dataframe
metadata_list = []

for i in range(50):
    folder = Path("../data/dfdc_train_part_" + str(i))
    metadata_file_path = folder/'metadata.json'
    metadata = pd.read_json(metadata_file_path).T

    metadata.reset_index(inplace=True)
    metadata.rename({'index':'fname'}, axis=1, inplace=True)
    
    metadata['directory'] =  str(folder) 
    
    metadata_list.append(metadata)
    
    
all_metadata = pd.concat(metadata_list)

In [3]:
all_metadata.tail()

,fname,label,split,original,directory
3129,pdooqxqfrm.mp4,FAKE,train,ikebomnsiq.mp4,../data/dfdc_train_part_49
3130,djjdcnhlma.mp4,FAKE,train,kudvvlgiff.mp4,../data/dfdc_train_part_49
3131,fgmbxfqoze.mp4,REAL,train,NaN,../data/dfdc_train_part_49
3132,cywebjaezn.mp4,REAL,train,NaN,../data/dfdc_train_part_49
3133,ohmkmcfxul.mp4,FAKE,train,hysmzkqsdl.mp4,../data/dfdc_train_part_49


In [15]:
paths = [str(Path(f).name).replace('_FAKE', '') for f in (glob.glob("../data/hard_data/*.mp4"))]

In [19]:
fake_files = ['aaiqsitvnd.mp4', 'afvlrevses.mp4', 'azqycoiiuk.mp4', 'aawkmrlilr.mp4', 'bjxzcaifpw.mp4',
              'cjstqmiyud.mp4', 'ajxpaiqcee.mp4', 'yumeecupaw.mp4', 'bbbmfffsad.mp4', 'bbaeewtqei.mp4',
               'aafcgzwvmy.mp4', 'dwqhiwmswx.mp4', 'cczisiwrmd.mp4', 'danyfkofxk.mp4', 'aetpnobkcv.mp4',
               'agswjtuhss.mp4', 'ctjewcqxee.mp4', 'baxdvqglnu.mp4', 'houzrrfbqg.mp4', 'acfaxepklq.mp4',
               'tfaxqkuhoh.mp4', 'cmoqplbifs.mp4', 'aihvsjluzl.mp4', 'aazkzzkbns.mp4', 'ahqausishe.mp4',
               'atwmgvaauf.mp4', 'aafezqchru.mp4', 'aetbyrujti.mp4', 'dqqwmmwlbg.mp4', 'zuweqkkudv.mp4',
               'cathvygbkb.mp4', 'bgkmgallha.mp4', 'alhsvhumdw.mp4', 'bbfewvawtu.mp4', 'dkrvorliqc.mp4',
               'abkxlatant.mp4', 'ajnzuiktuo.mp4', 'bdbryopkaa.mp4', 'xnmmweqmdt.mp4', 'bvhzopqkek.mp4',
               'aaqaifqrwn.mp4', 'bsprlfyyyc.mp4', 'agbfymrrhv.mp4', 'aiuvaldnqj.mp4', 'cmgmhevsmr.mp4',
               'aabkwhhgwv.mp4', 'djjafmdtxy.mp4', 'zwwvpycsis.mp4', 'dmzbzcjsrg.mp4', 'efghsdmasb.mp4',
               'coulkppulq.mp4', 'adtovspidj.mp4', 'arqhhrzzfe.mp4', 'cjetyhnwpr.mp4', 'akqjhilhtc.mp4',
               'alddoocxqo.mp4', 'aahzjreawd.mp4', 'zvhdwittmk.mp4', 'avfoukjslp.mp4', 'abteztchqz.mp4',
               'abfvpzjkwr.mp4', 'aakkdgsmvl.mp4', 'cdyoiiuszk.mp4', 'beshruwzkt.mp4', 'aakjukgjme.mp4',
               'aqgkloqxld.mp4', 'hfkwogkzid.mp4', 'avocsaizhx.mp4', 'ammkabwonw.mp4', 'avcnyamvrb.mp4',
               'anrlivgkfq.mp4', 'cnbtegsjlx.mp4', 'cbulmphxfg.mp4', 'acgyspwyqr.mp4', 'abhdaoikio.mp4',
               'azivnorcbt.mp4', 'agsrndrnjg.mp4', 'aemldzfofe.mp4', 'aaeucwtkdx.mp4', 'cqnutosgsm.mp4',
               'ackyrwxeew.mp4', 'aagundkpoh.mp4', 'bzqemivbgx.mp4', 'akaqyxcfep.mp4', 'bnbxarxyqk.mp4',
               'dywkdfclzy.mp4', 'aqfeqkckjs.mp4', 'azpamhpoyy.mp4', 'aoclawrydd.mp4', 'ahpaydsovz.mp4',
               'ahencqpiin.mp4', 'aaaoqepxnf.mp4', 'caqderfjql.mp4', 'aawhvztjdt.mp4', 'ayclhmjajg.mp4',
               'bmjmjmbglm.mp4', 'bacnffbyky.mp4', 'abhgyltuqg.mp4', 'azylwizjmc.mp4', 'asgzesadhh.mp4']

len(fake_files)

100

We would like to build up a training dataset using these and other videos. We currently have 100 fake videos.

We would like:

- 100 Fake Videos
- 100 Real Videos corresponding to the fake videos

In [20]:
real_files = []

for file in fake_files:
    
    row = all_metadata.loc[all_metadata['fname'] == file]
    original = row['original'].iloc[0]
    
    real_files.append(original)

In [23]:
len(real_files)

100

## Generate Image Dataset

We want to take all of these videos, break them apart into frames and take some of the frames and create an image dataset with them.

In [26]:
def read_frame_as_size(video_path, size=(128, 128)):
    capture = cv2.VideoCapture(str(video_path))
    ret, frame = capture.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, size)
    capture.release()
    return frame


def read_frame(video_path):
    capture = cv2.VideoCapture(str(video_path))
    ret, frame = capture.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    capture.release()
    return frame


def read_all_frames(video_path):
    capture = cv2.VideoCapture(str(video_path))
    all_frames = []
    ret = True
    while True:
        ret, frame = capture.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            all_frames.append(frame)
        else:
            break

    capture.release()
    return all_frames


def read_random_frame(video_path):
    
    capture = cv2.VideoCapture(str(video_path))
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))    
    random_frame = int(random.random() * frame_count)
    # Set to read specific frame
    capture.set(cv2.CAP_PROP_POS_FRAMES, random_frame)
    ret, frame = capture.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    capture.release()
    return frame

In [27]:
retinaFace = EasyRetinaFace()

Loading pretrained model from Pytorch_Retinaface/weights/Resnet50_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:456


In [41]:
def create_images_from_videos(files, suffix, folder='train', num_frames=10):
    for file in tqdm(files):
        row = all_metadata.loc[all_metadata['fname'] == file].iloc[0]

        video_path = row['directory'] + "/" + row['fname']
        
        for i in range(num_frames):
            random_frame = read_random_frame(video_path)

            # Face Detection
            detections = retinaFace.detect(random_frame)

            if len(detections) > 0:
                x_min, y_min, x_max, y_max, prob = detections[0]

                #Crop the face
                raw_crop = random_frame[int(y_min):int(y_max), int(x_min):int(x_max)]

                #Save to disk
                im = Image.fromarray(raw_crop)
                im.save("../data/" + folder + "/" + row['fname'] + "_" + str(i) + "_" + suffix +".png")    
            else:
                print("Couldn't find image for", video_path)

In [38]:
len(fake_files), len(real_files)

(100, 100)

In [39]:
train_fake_files = fake_files[:80]
train_real_files = real_files[:80]

val_fake_files = fake_files[80:]
val_real_files = real_files[80:]

In [40]:
# Make training files
os.makedirs('../data/train', exist_ok=True)

create_images_from_videos(train_fake_files, suffix="FAKE", folder="train", num_frames=10)
create_images_from_videos(train_real_files, suffix="REAL", folder="train", num_frames=10)

../data/dfdc_train_part_36/aaiqsitvnd.mp4
../data/dfdc_train_part_46/afvlrevses.mp4
../data/dfdc_train_part_3/azqycoiiuk.mp4
../data/dfdc_train_part_41/aawkmrlilr.mp4
../data/dfdc_train_part_45/bjxzcaifpw.mp4
../data/dfdc_train_part_35/cjstqmiyud.mp4
../data/dfdc_train_part_2/ajxpaiqcee.mp4
../data/dfdc_train_part_48/yumeecupaw.mp4
../data/dfdc_train_part_29/bbbmfffsad.mp4
../data/dfdc_train_part_2/bbaeewtqei.mp4
../data/dfdc_train_part_31/aafcgzwvmy.mp4
../data/dfdc_train_part_26/dwqhiwmswx.mp4
../data/dfdc_train_part_7/cczisiwrmd.mp4
../data/dfdc_train_part_32/danyfkofxk.mp4
../data/dfdc_train_part_47/aetpnobkcv.mp4
../data/dfdc_train_part_44/agswjtuhss.mp4
../data/dfdc_train_part_2/ctjewcqxee.mp4
../data/dfdc_train_part_43/baxdvqglnu.mp4
../data/dfdc_train_part_2/houzrrfbqg.mp4
../data/dfdc_train_part_48/acfaxepklq.mp4
../data/dfdc_train_part_7/tfaxqkuhoh.mp4
../data/dfdc_train_part_38/cmoqplbifs.mp4
../data/dfdc_train_part_32/aihvsjluzl.mp4
../data/dfdc_train_part_46/aazkzzkbns.mp4

../data/dfdc_train_part_36/frehtgxiln.mp4
../data/dfdc_train_part_46/knbodjyiew.mp4
../data/dfdc_train_part_3/nxylqxbxnz.mp4
../data/dfdc_train_part_41/ekbcrbtmmc.mp4
../data/dfdc_train_part_45/fwisndrpzm.mp4
../data/dfdc_train_part_35/tfddwvpqxx.mp4
../data/dfdc_train_part_2/ztyvglkcsf.mp4
../data/dfdc_train_part_48/rmldoykgzt.mp4
../data/dfdc_train_part_29/quyzfrphft.mp4
../data/dfdc_train_part_2/rfukeoxpqu.mp4
../data/dfdc_train_part_31/nebevwudhr.mp4
../data/dfdc_train_part_26/tvdlpqwnzl.mp4
../data/dfdc_train_part_7/hkplvhwibp.mp4
../data/dfdc_train_part_32/rzorqxynsl.mp4
../data/dfdc_train_part_47/pldgpdtzwo.mp4
../data/dfdc_train_part_44/cabhgzgpba.mp4
../data/dfdc_train_part_2/abmjszfycr.mp4
../data/dfdc_train_part_43/zcjrdqfnsa.mp4
../data/dfdc_train_part_2/dmmvuaikkv.mp4
../data/dfdc_train_part_48/nthlmvjxac.mp4
../data/dfdc_train_part_7/vxoaanowge.mp4
../data/dfdc_train_part_38/uvnefxyqsy.mp4
../data/dfdc_train_part_32/vvqserbnug.mp4
../data/dfdc_train_part_46/mcehvkoidm.mp4

In [42]:
# Make validation files
os.makedirs('../data/val', exist_ok=True)
create_images_from_videos(val_fake_files, suffix="FAKE", folder="val", num_frames=10)
create_images_from_videos(val_real_files, suffix="REAL", folder="val", num_frames=10)